### Part1 Data acquisition
#### 1.OHLC(date)
#### 2.concat_csv(stock_id)
#### 3.fetch_data(startyear, startmonth,stock_id)
###### editor: Sam Chen 2021/07

In [2]:
#Package import
import os
import glob
import time
import datetime
import requests
import pandas as pd
from io import StringIO
from datetime import date

# Chart drawing
import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

#create new folder
def createFolder(directory): #directory means the file's name
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' + directory)
# Example
#createFolder('./5501/')
# Creates a folder in the current directory called data


In [4]:
def OHLC(date,stock_id):
    stock_id =str(stock_id)
    date = str(date)
    timestamp = str(time.time() * 1000 + 1000000)
    url = 'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date='+ date
    #https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20210702&st
    res = requests.get(url)
    
    #"日期","成交股數","成交金額","開盤價","最高價","最低價","收盤價","漲跌價差","成交筆數
    content = res.text.replace(':[[','],[').replace("]]","],[").split('],[')
    lines = list(filter(lambda l:(len(l.split('",')) < 10) & (len(l.split('",')) > 6)))
    content = "\n".join(lines)
    
    #Index column set
    df = pd.read_csv(StringIO(content),names=["日期","成交股數",
                                              "成交金額","開盤價","最高價","最低價",
                                              "收盤價","漲跌價差","成交筆數"])
    df = df.astype(str)
    
    #Change date format (108->2019)
    d = df['日期']
    for i in range(len(d)):
        df.iloc[i]=d.iloc[i].replace(d.iloc[i][0:3], str(int(d.iloc[i][0:3]) + 1911))
        df_dt=pd.to_datetime(df.日期,format="%Y/%m/%d")
        df.set_index('日期',inplace=True)
    
    #Last Check
    df = df.apply(lambda s: s.str.replace(',', ''))
    df = df.apply(lambda s:pd.to_numeric(s, errors='coerce'))
    df = df[df.columns[df.isnull().all() == False]]
    
    #Change file directory to stock_id individual flile
    path = os.chdir('C:\\Users\\SC.210\\FinTech\\stock\\'+ stock_id)
    df.to_csv(date +'.csv',encoding='utf_8_sig')
    print(date +'.csv is downloaded!')

In [5]:
#Download from 2019~2020 OHLC data
def fetch_data(startyear, startmonth,stock_id):
    currentdate = datetime.datetime.today().strftime("%Y/%m/%d")
    currentyear = datetime.datetime.now().year
    currentmonth = datetime.datetime.now().month
    print('Current date: ' + str(currentdate))
    
    for i in range(startyear, currentyear+1):
        if (i != currentyear):
            for j in range(startmonth, 13):
                if j <10:
                    date = str(i)+'0'+str(j)+'01'
                    OHLC(date,stock_id)
                    time.sleep(3) #sleep for 5 secods avoid rate limiting
                else:
                    date = str(i)+str(j)+'01'
                    OHLC(date,stock_id)
                    time.sleep(3)
        else:
           for j in range(startmonth, currentmonth):
                if j <10:
                    date = str(i)+'0'+str(j)+'01'
                    OHLC(date,stock_id)
                    time.sleep(3)
                else:
                    date = str(i)+str(j)+'01'
                    OHLC(date,stock_id)
                    time.sleep(3)
        print('===========================')
        print('Download done!!')

In [1]:
import OHLC

def crawler(stock_id):
    stock_id = str(stock_id)
    os.chdir('C:\\Users\\SC.210\\FinTech\\stock') #Change the current working direct
    
    createFolder('./'+stock_id+'/') #Create new folder
    os.chdir('C:\\Users\\SC.210\\FinTech\\stock\\'+stock_id) #Change the current wor
    
    #Fetch data from TWSE (indiviual information)
    fetch_data(2010,1,stock_id)
    
    #Concat all the csv to signle file
    concat_csv(stock_id)
    
    print('Finish crawling stock: '+ stock_id)

ModuleNotFoundError: No module named 'main'

In [6]:
def concat_csv(stock_id):
    #Enter the path where store csv files
    path = 'C:/Users/SC.210/FinTech/stock/'+str(stock_id)
    os.chdir(path)
    
    #Use glob to match the pattern ‘csv’
    extension = 'csv'
    all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
    
    #Combine all files in the list
    combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
    
    #Export to csv
    combined_csv.to_csv( "tse_"+str(stock_id)+".csv", index=False, encoding='utf-8-s')

In [ ]:
crawer

### Part2 Workplace
#### 1.Download OHLC data from 2010~2020
#### 2.Visuzlize the trend and technical indicators

from csv import reader
os.chdir('C:\\Users\\SC.210\\FinTech') #Change the current working directory

Stock=[]
with open('stocknumber.csv','r',encoding='utf-8') as read_obj:
csv_reader = reader(read_obj)
for row in csv_reader:
Stock.append(row)

Stock[0] = ['1101']
Stock[150]

In [7]:
for i in range(101,150):
    s = str(Stock[i]).replace("['","").replace("']","")
    crawler(s)
    time.sleep(8)

NameError: name 'Stock' is not defined

In [ ]:
df = pd.read_csv(r'C:\Users\SC.210\FinTech\2436\tse_2436.csv')
fig = go.Figure(data=[go.Candlestick(x=df['日期'],
                                     open=df['開盤價'],
                                     high=df['最高價'],
                                     low=df['最低價'],
                                     close=df['收盤價'])])
fig.show()

### Part3 Technical Indicator
#### 1. Moving average
#### 2. Relative strength index(RSI)
#### 3. MACD

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Chart drawing
import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# Show charts when running kernel
init_notebook_mode(connected=True)

# Change default background color for all visualizations
layout=go.Layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(250,250,250,0.8)')
fig = go.Figure(layout=layout)
templated_fig = pio.to_templated(fig)
pio.templates['my_template'] = templated_fig.layout.template
pio.templates.default = 'my_template'

In [ ]:
os.chdir('C:\\Users\\SC.210\\FinTech\\2436')
df = pd.read_csv('tse_2436.csv')
#Moving average
df['EMA_9'] = df['收盤價'].ewm(9).mean().shift()
df['SMA_5'] = df['收盤價'].rolling(5).mean().shift()
df['SMA_10'] = df['收盤價'].rolling(10).mean().shift()
df['SMA_15'] = df['收盤價'].rolling(15).mean().shift()
df['SMA_30'] = df['收盤價'].rolling(30).mean().shift()
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['日期'], y=df.EMA_9, name='EMA 9'))
fig.add_trace(go.Scatter(x=df['日期'], y=df.SMA_5, name='SMA 5'))
fig.add_trace(go.Scatter(x=df['日期'], y=df.SMA_10, name='SMA 10'))
fig.add_trace(go.Scatter(x=df['日期'], y=df.SMA_15, name='SMA 15'))
fig.add_trace(go.Scatter(x=df['日期'], y=df.SMA_30, name='SMA 30'))
fig.add_trace(go.Scatter(x=df['日期'], y=df['收盤價'], name='Close', opacity=0.2))
fig.show()

In [ ]:
#Relative Strength index(RSI)
def relative_strength_idx(df, n=14):
    close = df['收盤價']
    delta = close.diff()
    delta = delta[1:]
    pricesUp = delta.copy()
    pricesDown = delta.copy()
    pricesUp[pricesUp < 0] = 0
    pricesDown[pricesDown > 0] = 0
    rollUp = pricesUp.rolling(n).mean()
    rollDown = pricesDown.abs().rolling(n).mean()
    rs = rollUp / rollDown
    rsi = 100.0 - (100.0 / (1.0 + rs))
    return rsi

df['RSI'] = relative_strength_idx(df).fillna(0)

fig = go.Figure(go.Scatter(x=df['日期'], y=df.RSI, name='RSI'))
fig.show()

In [ ]:
#MACD
EMA_12 = pd.Series(df['收盤價'].ewm(span=12, min_periods=12).mean())
EMA_26 = pd.Series(df['收盤價'].ewm(span=26, min_periods=26).mean())
df['MACD'] = pd.Series(EMA_12 - EMA_26)
df['MACD_signal'] = pd.Series(df.MACD.ewm(span=9, min_periods=9).mean())

fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=df['日期'], y=df['收盤價'], name='收盤價'), row=1, col=1)
fig.add_trace(go.Scatter(x=df['日期'], y=EMA_12, name='EMA 12'), row=1, col=1)
fig.add_trace(go.Scatter(x=df['日期'], y=EMA_26, name='EMA 26'), row=1, col=1)
fig.add_trace(go.Scatter(x=df['日期'], y=df['MACD'], name='MACD'), row=2, col=1)
fig.add_trace(go.Scatter(x=df['日期'], y=df['MACD_signal'], name='Signal line'), row=2 , col=1)
fig.show()

In [ ]:
def technical_index(stock_id):
    os.chdir('C:\\Users\\SC.210\\FinTech\\stock\\'+str(stock_id))
    df = pd.read_csv('tse_'+str(stock_id)+'.csv')
    
    #Moving average
    df['EMA_9'] = df['收盤價'].ewm(9).mean().shift()
    df['SMA_5'] = df['收盤價'].rolling(5).mean().shift()
    df['SMA_10'] = df['收盤價'].rolling(10).mean().shift()
    df['SMA_15'] = df['收盤價'].rolling(15).mean().shift()
    df['SMA_30'] = df['收盤價'].rolling(30).mean().shift()
    
    #Relative Strength index(RSI)
    def relative_strength_idx(df, n=14):
        close = df['收盤價']
        delta = close.diff()
        delta = delta[1:]
        pricesUp = delta.copy()
        pricesDown = delta.copy()
        pricesUp[pricesUp < 0] = 0
        pricesDown[pricesDown > 0] = 0
        rollUp = pricesUp.rolling(n).mean()
        rollDown = pricesDown.abs().rolling(n).mean()
        rs = rollUp / rollDown
        rsi = 100.0 - (100.0 / (1.0 + rs))
        return rsi
    
    df['RSI'] = relative_strength_idx(df).fillna(0)
    
    #MACD
    EMA_12 = pd.Series(df['收盤價'].ewm(span=12, min_periods=12).mean())
    EMA_26 = pd.Series(df['收盤價'].ewm(span=26, min_periods=26).mean())
    df['MACD'] = pd.Series(EMA_12 - EMA_26)
    df['MACD_signal'] = pd.Series(df.MACD.ewm(span=9, min_periods=9).mean())
    df.to_csv( "tse_"+str(stock_id)+".csv", index=False, encoding='utf-8-sig')

In [ ]:
technical_index(1530)

In [ ]:
for i in range(1,101):
    s = str(Stock[i]).replace("['","").replace("']","")
    technical_index(s)
    time.sleep(2)